# Using statistical tests — correlation (optional)

This Notebook will show some examples of using statistical tests to get qualitative results from questions asked of the accidents dataset.

*To be in any way meaningful, rather than misleading, statistical methods need to be treated with care and used in an appropriate way. __This module is not intended to teach you statistical methods.__*

*The examples provided here should be regarded primarily as examples of how to apply powerful statistics based Python packages to datasets, rather than examples of how to "do statistics".*

Once again, we'll be using analysing data grabbed from a MongoDB database into *pandas* dataframes, with all the risks that implies from the point of view of polluting the results data with artefacts arising from the re-presentation of of the potentially irregular results documents in regular, tabular form. The statistical analyses will be performed using functions from the `scipy.stats` package.

In [ ]:
# Standard imports

import pandas as pd
import numpy as np
import scipy.stats

import matplotlib.pyplot as plt
import seaborn as sns

## Setting up the document database 

In the notebooks for parts 14, 15 and 16, you will be using a document database to manage data. As with the relational database you looked at in previous sections, the data in the database is *persistent*. The document database, MongoDB, is described as "NoSQL" to reflect that it does not use the tabular format of the relational database to store data. However, many of properties of a formal RDBMS apply to MongoDB, including the need to connect to the database server.

As with PostgreSQL, the MongoDB database server runs independently from the Jupyter notebook server. To interact with it, you need to set up an explicit connection.

### Setting your database credentials

In order to work with a database, we need to create a *connection* to the database. A connection allows us to manipulate the database, and query its contents (depending on what usage rights you have been granted). For the SQL notebooks in TM351, the details of your connection will depend upon whether you are using the OU-hosted server, accessed via [tm351.open.ac.uk](https:tm351.open.ac.uk), or whether you are using a version hosted on your own computer, which you should have set up using either Vagrant or Docker.

To set up the connection, you need a login name and a pasword. we will use the variables `DB_USER` and `DB_PWD` to hold the user name and password respectively that you will use to connect to the database. Run the appropriate cell to set your credentials in the following cells.

#### Connecting to the database on [tm351.open.ac.uk](https:tm351.open.ac.uk)

If you are using the Open University hosted server, you should execute the following cell, using your OUCU as the value of `DB_USER`, and the password you were given at the beginning of the module. Note that if the cell is in RAW NBconvert style, you will need to change its type to Code in order to execute it.

The variables `DB_USER` and `DB_PWD` are strings, and so you need to put them in quotes.

In this case, note that the connection string contains an additional option at the end: `?authsource=user-data`. For the MongoDB setup that we are using here, this option tells Mongo where to look for the authentication database.

#### Connecting to the database on a locally hosted machine

If you are running the Jupyter server on your own machine, via Docker or Vagrant, you should execute the following cell. Note that if the cell is in RAW NBconvert style, you will need to change its type to Code in order to execute it.

Note that the locally hosted versions of the environment give you full administrator rights, which is why you do not need to specify a user name or password. Obviously, this would not generally not be granted on a multi-user database, unless you are the database administrator.

### Connecting to the database

We can now set up a connection to the database. As with PostgreSQL, we use a connection string:

In [ ]:
print(MONGO_CONNECTION_STRING)

The connection string is made up of several parts:

- `mongodb` : tells `pymongo` that we will use MongoDB as our database engine
- Your user name and (character escaped) password, separated by a colon if you are using the remote server. If you are using a local server, you will be logged on as an adminstrator, and do not need to specify a name or password.
- `localhost:27017` : the port on which the database engine is listening.
- A reference to the authentication file (`?authsource=user-data`), if you are using the remote server.

We now connect to the database with a `pymongo.MongoClient` object.

In [ ]:
from pymongo import MongoClient

In [ ]:
mongo_client=MongoClient(MONGO_CONNECTION_STRING)

You should now be connected to the MongoDB database server.

## The accidents database

The accidents database takes a long time to set up, so we have already imported it into a MongoDB database so that you can work with it. Note that on the remote VCE, the database is read-only, so you will not be able to alter its contents, although you can copy the contents into your own database space as discussed in the previous MongoDB notebooks, and alter that.

The cells in the earlier section, Setting up the document database, put the name of the accidents database into the variable `ACCIDENTS_DB_NAME`. Use this value to set up the connection to the `accidents` database and collections within it:

In [ ]:
accidents_db=mongo_client[ACCIDENTS_DB_NAME]

We can look at the names of the collections in the database:

In [ ]:
accidents_db.list_collection_names()

We will introduce some of the different collections in the rest of the materials, but let's start with the `accidents` collection:

In [ ]:
accidents_collection=accidents_db['accidents']

This collection contains information on individual accidents. We can see how many examples it contains with the `.count_documents()` method:

In [ ]:
accidents_collection.count_documents({})

Set a larger plot size than the default:

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

## Comparing the number of casualties and vehicles
in Notebook `14.2 Introduction to accidents`, we presented a simple visual analysis comparing the number of casualties with the number of vehicles across a large number of accidents.

The following code retrieves the relevant data from the database and puts it into a long form with two relevant columns: the number of casualties and the number of vehicles. Each row corresponds to a separate accident.

In [ ]:
# Build a DataFrame, one row for each accident
cas_veh_df = pd.DataFrame(accidents_collection.find({'Local_Authority_(Highway)': 'E06000042'},
                                                 ['Number_of_Casualties', 'Number_of_Vehicles']))
cas_veh_df.head(5)

As we noted before, the regression line is a simple best fit line, calculated over the raw, item level accident data.

In [ ]:
regressionline = scipy.stats.linregress(cas_veh_df['Number_of_Casualties'],
                                        cas_veh_df['Number_of_Vehicles'])

# The regression line is of the form y = m x + b
m = regressionline[0]
b = regressionline[1]
(m, b)

We can also force the data into a different long form with three columns: the number of casualties, the number of vehicles, and the count of accidents with that *(casualty, vehicle)* combination. The data in this format can then easily be plotted as a bubble chart.

In [ ]:
# Count the number of each severity
cas_veh_crosstab = pd.crosstab(cas_veh_df['Number_of_Casualties'],
                               cas_veh_df['Number_of_Vehicles'])
# Reshape
cas_veh_crosstab = cas_veh_crosstab.stack().reset_index()
cas_veh_crosstab.head(3)

The following chart demonstrates how the count data and the regression line can be plotted on the same chart using a matplotlib chart:

In [ ]:
plt.scatter(cas_veh_crosstab['Number_of_Casualties'], 
            cas_veh_crosstab['Number_of_Vehicles'],
            s = np.sqrt(cas_veh_crosstab[0])*50,
            alpha=0.5
            )

x = np.linspace(0, 30, 20)
plt.plot(x, m*x + b)

plt.xlabel('Number of casualties')
plt.ylabel('Number of vehicles')
plt.show()

Alternatively, we can create the chart directly from the original line item dataset and let the chart do the counting for us:

In [ ]:
# The best fit/linear regression chart may take some time to calculate
sns.lmplot(x="Number_of_Casualties", y="Number_of_Vehicles", data=cas_veh_df,
           x_jitter=0.2, y_jitter=0.2, scatter_kws={'s':1}
          )

# Set the x axis limits (None means set automatically)
plt.xlim(0, None);

Whilst he best fit line might suggest there is an increasing relationship between the number of casualties and the number of vehicles, the scatter / bubble marks on the chart suggest otherwise.

So is there a better, mathematical, way of identifying whether there is a relationship or not?

## Measuring Correlation Coefficients

To measure the strength of the relationship between two sets of numbers, we can use a statistical measure know as a *correlation coefficient*.

### Pearson's *r* - "The" Correlation Coefficient

`Pearson's r`, also know as Pearson's correlation coefficent, or even as *the* correlation coefficient, is a number ion the range -1..+1 that indicates the relationship between two lists of numbers. If the value is +1, the numbers increase in value at the same rate in each list; if the number is -1, the numbers in one list grow at the same absolute rate as the numbers decrease in the other list. If the coefficient is zero, there is no relationship in the way the numbers go up and down in each list.

*This is incorrectly referred to in the VLE materials as Pearson's $R^2$, although it may be referred to as Pearson's R.*

The `pearsonr` function calculates Pearson's *R* value of correlation. The function takes two lists of numbers, of equal lengths. The Pearson's *R* function looks at the values at the same index in both lists and finds how the values in one column vary with respect to the other column. 

Note that we have to give each accident on its own row: if there are 145,000 accidents, the `pearsonr` function must be passed lists with 145,000 items.

Recall that values near +1 show good positive correlation, values near -1 show good negative correlation, and values near 0 show no particular correlation. The `scipy` function also returns a second value, the *(2-tailed) p* value of the result, which gives a measure of the so-called *"statistical significance"* of the result. In this case, the *p* value is a measure of the likelihood that the effect the correlation coefficient measures does actually exist (i.e. whether that degree of correlation really is how correlated the lists are) or whether the degree of correlation measured is as likely to have occurred in two random lists of numbers.

*(Generally, the closer the *p* value is to zero, the more unlikely it is that the effect indicated by the correlation coefficient does not actually exist; which is to say, the less likely it is that the measured effect does not hold, or the more likely it is that the reported effect does hold.)*

The function is applied simply by passing in the two aligned lists of data whose correlation we wish to measure:

In [ ]:
(r, p) = scipy.stats.pearsonr(cas_veh_df['Number_of_Casualties'],
                              cas_veh_df['Number_of_Vehicles'])

print('r: {r}, p: {p}'.format(r=r, p=p))

This result shows a small, positive correlation (`r`) with a very small *p* (`p`) value.

In other words, there's not much correlation, and the result is statistically significant (highly unlikely that the effect measured occurred by chance). This means we can reject the the null hypothesis that the number of casualties in an accident is unrelated to the number of vehicles.

Looking at the data, it seems to be a result that most accidents result in very few casualties, and the accidents with the most casualties have few vehicles.

Can you think of a reason for this?

*Your thoughts here.*

*Learn more about this in the optional notebook `Using statistical tests — regression (optional)`.*

## Spearman's Rank Correlation Coefficient, $\rho$

Ages of people in the accidents dataset are stored as bands, not continuous values. This means that correlations between them must use a different correlation measure, Spearman's *rank* correlation coeffient, or Spearman's $\rho$ ("Spearman's rho" (pronounced: row as in in boat).

This measure again takes two lists of values, but rather than calculating the simple Pearson correlation coefficient, it first ranks the values in each list, and then measures the correlation between the ranked values.

Similar to the Pearson function above, the `scipy.stats.spearmanr()` function takes two parameters, each a list of values for the two variables being compared, to calculate the rank correlation between the lists.

Spearman's rank correlation is useful when we can order — that is, *rank* — items, but where the quantity we are ranking may not itself be distributed in a convenient way or may not provide numeric values with suitable intervals that we could use as part of Pearson's r calculation.

One such example might be where we have age bands over different ranges, such as `[under18, 18-25, 26-40, 41-65, 65+]`. In this case we might use the rank of each group in the ordered list of groups for the purpose of correlation calculations, rather than try to associate an a single specific age with each group.

In the *accidents* dataset, an age range field field is referred to from the `'Age_Band_of_Driver'` and `'Age_Band_of_Casualty'` fields.

The `Age_Band_of_Driver` labels are defined as:

In [ ]:
labels=accidents_db['labels']

In [ ]:
labels.find_one({'label': 'Age_Band_of_Driver'})['codes']

and the `Age_Band_of_Casualty` codes we have met before:

In [ ]:
labels.find_one({'label': 'Age_Band_of_Casualty'})['codes']

*A `-1` null code also exists for situations where the age is missing or not known but this value is not defined in the `labels` collection.*

The age range bands have a natural rank order, but may have arbitrary code values. (In actual fact, the code values for these groups (the index values) *are* meaningfully numerically ordered, but let's imagine for now that they aren't.)

How might we go about identifying whether or not there is any correlation between the ages of a driver and a passenger, at least according to the accidents database?

*Add your thoughts here...*

One way in to exploring this question might be to generate a table where each row has two values: one for the age band of the driver, and one for the age band of a passenger.

Calculating Spearman's rank correlation coefficient across these two columns should identify whether a relationship holds between them; for example, as far as vehicles involved in accidents go, do young drivers have young passengers, are old drivers associated with old passengers, etc.

The next few setions and activities are used to construct just such a dataframe, before we move on to calculating the rank correlation between the age range values.

## Chi-squared ($\chi^2$) example 1: hypothetical voting intention
This is the same example as used in the VLE teaching material, based on gender related voting across different political parties, showing how the chi-squared ($\chi^2$) statistic is calculated.

Let's start by creating a DataFrame from a simple `dict`, where each entry is a column in the DataFrame. The key is the column name, the value is the items in the column. Each set of column values is itself a `dict`, with one key for each index entry and the value being the contents of that cell in the DataFrame.

In [ ]:
actual_survey_results = pd.DataFrame({'Conservative': {'Men': 170, 'Women': 220},
                                      'Labour': {'Men': 240, 'Women': 190},
                                      'Other': {'Men': 80, 'Women': 100}})
actual_survey_results

The ($\chi^2$) test (sometimes also referred to as *Pearson's chi-squared test*) is a test applied to a so-called a contingency table to compare actual and expected counts between various observed categories.

We could find the expected counts manually, or we could use the `scipy.stats.contingency.expected_freq()` function to do it for us. Note that this returns an array, rather than a DataFrame, but it's the same shape as the original DataFrame (that is, is has the same number of rows and columns, arranged in the same way).

Here's how to use the `scipy.stats` function:

In [ ]:
scipy.stats.contingency.expected_freq(actual_survey_results)

We can then wrap this in a DataFrame labeled using the same column and index values of the source DataFrame:

In [ ]:
_array = scipy.stats.contingency.expected_freq(actual_survey_results)
pd.DataFrame(_array, columns=actual_survey_results.columns).set_index(actual_survey_results.index)

Alternatively, we could create our own function:

In [ ]:
def expected_of_df(actual_df):
    df = pd.DataFrame(
        {c: 
         {r: actual_df[c].sum() * actual_df.loc[r].sum() / actual_df.sum().sum()
                  for r in actual_df[c].index} 
              for c in actual_df})
    # Fix the order of columns and rows
    df = df[actual_df.columns]
    df = df.reindex(actual_df.index)
    return df

And then check that we get a similar result:

In [ ]:
expected_survey_results = expected_of_df(actual_survey_results)
expected_survey_results

As we're using a table of several rows and columns, we use the `scipy.stats.chi2_contingency()` function to find the $\chi ^ 2$ statistic and the *p* value. 

Note that the function returns $\chi ^ 2$, the *p* value, the number of degrees of freedom, and the matrix of expected frequencies. We're generally after just the second returned value, the *p* value.

In [ ]:
scipy.stats.chi2_contingency(actual_survey_results)

In [ ]:
chi2, p, _, _ = scipy.stats.chi2_contingency(actual_survey_results)
'chi2: {}, p: {}'.format(chi2, p)

The *p* value of 0.0009, a very small value, suggests that the observed values are highly unlikely to have been the result of chance, and means that we can reject the null hypothesis that voting intention is independent of gender: for this example, it seems that we *can* say that men and women vote differently.

If we adjust the numbers slightly, we can get a very different result.

In [ ]:
actual_survey_results_2 = pd.DataFrame({'Conservative': {'Men': 170, 'Women': 220},
                      'Labour': {'Men': 220, 'Women': 210},
                      'Other': {'Men': 80, 'Women': 100}})
actual_survey_results_2

In [ ]:
expected_survey_results_2 = expected_of_df(actual_survey_results_2)
expected_survey_results_2

In [ ]:
chi2, p, _, _ = scipy.stats.chi2_contingency(actual_survey_results_2)
chi2, p

The *p* value of 0.07 means that there is a 7% chance that the observed values may have occurred by chance, which is more than the 5% (1 in 20) chance that we typically use to to decide whether something is "statistically significant";  that is, we *cannot* reject (at the 5% level) the null hypothesis that voting intention is independent of gender: for this modified example, we *can't* say that men and women vote differently.

## Chi square example 2: accident frequency by day of week
Let's look to see if more accidents occur on different days of the week. That is, from the observed number of accidents, how likely is it that the observed pattern of accidents occurred by chance, or does it seem to be the case that more accidents appear on some days than others?

We'll start by counting the number of accidents that occurred on each day of the week:

In [ ]:
# Build a DataFrame, one row for each accident
count_by_day_df = pd.DataFrame(accidents_collection.find({}, ['Day_of_Week']))

# Find counts for each day
count_by_day_ss = count_by_day_df['Day_of_Week'].value_counts()

# Reorder by day of week, add labels.
day_of_week_labels = labels.find_one({'label': 'Day_of_Week'})['codes']

count_by_day_ss.sort_index(inplace=True)
count_by_day_ss.index = [day_of_week_labels[str(r)] for r in count_by_day_ss.index]

count_by_day_ss

And visualise the result to get a preliminary, gut reaction feel for it:

In [ ]:
count_by_day_ss.plot(kind='bar', color='royalblue');

There look to be differences, but are they significant?

We run into a slight problem here, though: the `scipy.stats.contingency.expected_freq` and `scipy.stats.chi2_contingency` functions we used in the voting example above assume the data is in a table of at least two rows and at least two columns.

Whilst the calculations do run without error, the results are not informative.

For example, the expected frequencies are the same as the actual ones:

In [ ]:
scipy.stats.contingency.expected_freq(count_by_day_ss)

And the $\chi^2$ value is 0, with a *p*-value of 1.

In [ ]:
chi2, p, _, _ = scipy.stats.chi2_contingency(count_by_day_ss)
'chi2: {}, p: {}'.format(chi2, p)

This means we have to use rather less convenient functions to calculate the $\chi^2$ and _p_ values.

First, we explicitly find the expected values:

In [ ]:
expected_count_by_day_ss = pd.Series(count_by_day_ss.sum() / len(count_by_day_ss), 
                                  index=count_by_day_ss.index)
expected_count_by_day_ss

Then we use the `scipy.stats.chisquare()` function to find the test results. Note that the *p* value is nicely labelled for us.

In [ ]:
scipy.stats.chisquare(count_by_day_ss, expected_count_by_day_ss)

The *p* value of zero shows that this is a statistically significant result — the observed values are highly unlikely to have been the result of chance — and from that we might conclude that the varying number of accidents by day is significant.

### Activity 1

As well as effects by day of week, we might expect that accidents are more likely in bad weather. We might also expect that the weather conditions are likely to affect different roads differently, with bad conditions on high-speed roads having more of an impact on accident likelihood than low-speed (typically urban) roads (perhaps because reaction times are more compromised by the higher speed conditions or perhaps because of anticipated larger traffic volumes).

If the weather affects all roads equally, we would expect to see the proportions of accidents in different weather conditions to be the same for different road speed limits. 

Use a chi-squared test to determine if the proportion of accidents in different weather conditions is independent of road speed.

Note that this activity will require several stages, following the pattern above: finding the values for the different ranges of `Weather_Conditions` (from inspection of the `label_of` dictionary), extracting the data from the database into a DataFrame and calculating the expected values for each combination of speed limit and weather and finally calculating the chi-squared *p* value.

We have broken the task down into several subtasks.

What are the weather types?


In [ ]:
# Enter your code in this cell

Retrieve the weather and speed data for each accident.


In [ ]:
# Enter your code in this cell

Count the number of accidents for combination of speed and weather, and label the rows and columns with meaningful labels.

In [ ]:
# Enter your code in this cell

Calculate the chi-squared value.

In [ ]:
# Enter your code in this cell

*Comment here on the $\chi^2$ value. What does it appear to tell you?*

Write your comments in this cell

Plot the data to see whether any possibly effects or features jump out at you.

In [ ]:
# Enter your code in this cell

#### Our solution

To reveal our solution, run this cell or click on the triangle symbol on the left-hand side of the cell.

First, we need to identify the different recorded weather conditions:

In [ ]:
label = 'Weather_Conditions'

weather_conditions_labels = labels.find_one({'label': label})['codes']
weather_conditions_labels

Next we need to retrieve the weather and speed data for each accident:

In [ ]:
# Build a DataFrame, one row for each accident
speed_by_weather_df = pd.DataFrame(accidents_collection.find({}, ['Speed_limit', 'Weather_Conditions']))
speed_by_weather_df.head()

And count the number of accidents for each speed and weather combination:

In [ ]:
# Count the number of each severity
speed_by_weather_df = pd.crosstab(speed_by_weather_df['Speed_limit'],
                                  speed_by_weather_df['Weather_Conditions'])

speed_by_weather_df

Label the rows and columns appropriately:

In [ ]:
speed_by_weather_df.columns = [weather_conditions_labels[str(w)] for w in speed_by_weather_df.columns]
speed_by_weather_df

Calculate the $\chi^2$ value:

In [ ]:
(chi2, p, _, _) = scipy.stats.chi2_contingency(speed_by_weather_df)

print(f'Chi-squated: {chi2}, p: {p}')

The very small *p* value shows that this is a significant result: it seems likely that chance does not explain the observed distribution and so weather conditions do seem to affect accident rates differently on different roads.

Note that the chi-squared test doesn't tell us anything about *how* the weather conditions affect accident rates, only that they do. Visualising the data may help us to see some sort of effect that we could then check using further statistical tests.

In [ ]:
ax = speed_by_weather_df.plot(kind='bar')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5));

Hmmm... it's hard to see from that what's going on, but if we view the data using a log scale on the y axis (add the argument `logy=True` to the `.plot()` function) or limit the y-axis range (for example, pass `ylim=(0,1500)`) we can see more and different detail. To me, the shape of distribution of accidents by weather type does look sort of similar on each road type... More detailed investigation needed, methinks...

#### End of Activity 1

-----------------------------------

## What next?

This is one of two optional notebooks. If you have time available, consider reviewing the other optional notebook. Otherwise, return to the module materials now.